In [1]:
import pandas as pd
import yaml
import numpy as np

Trying to find a way to quickly parse an SRA Metadata table into a YAML sample listing for input to my ATAC pipeline

In [2]:
data = pd.read_csv("SraRunTable.txt-3.csv")

In [3]:
data_small = data[["Run", "Cell_type", "time_point", "treatment", 'genotype/variation']]
data_small = data_small.fillna('None')
data_small['condition'] = data_small['genotype/variation'] + "_" + data_small['Cell_type'] + "_" + data_small["treatment"] + "_" + data_small["time_point"]

In [4]:
data_small.condition = data_small.condition.str.replace(" ", "_").str.replace("+", "")

In [5]:
data_multi = data_small.sort_values("condition").set_index(["condition", "Run"])

In [14]:
d = {}
for condition, sample in data_multi.index:
    if condition not in d.keys():
        d[condition] = []
    d[condition].append(sample)

In [20]:
config = {'samples' : d}
with open('config.yml', 'a') as f:
    f.write(yaml.dump(config, default_flow_style=False))

In [24]:
l = range(0,6)

In [27]:
len(l)

6